In [131]:
# Get guns data
import pandas as pd
gun_death = pd.read_csv("guns.csv", index_col=0)
gun_death[:1]

,year,month,intent,police,sex,age,race,hispanic,place,education
1,2012,1,Suicide,0,M,34.0,Asian/Pacific Islander,100,Home,4.0


In [132]:
# gun death by year
death_by_year = gun_death.groupby(["year"])["year"].count()
death_by_year

year
2012    33563
2013    33636
2014    33599
Name: year, dtype: int64

In [133]:
#gun death by year and month
import datetime

gun_death["day"] = 1
gun_death["datetime"] = pd.to_datetime(gun_death[["year", "month", "day"]])
gun_death[:1]

,year,month,intent,police,sex,age,race,hispanic,place,education,day,datetime
1,2012,1,Suicide,0,M,34.0,Asian/Pacific Islander,100,Home,4.0,1,2012-01-01


The number of death is not significantly different if we look only by year. 
If we specify the number of death by year and month we cannot see any significant patterns.

In [134]:
#death by race and sex
sex_group = gun_death[["sex"]].groupby("sex").size()
race_group = gun_death[["race"]].groupby("race").size()
print(sex_group, race_group)

sex
F    14449
M    86349
dtype: int64 race
Asian/Pacific Islander             1326
Black                             23296
Hispanic                           9022
Native American/Native Alaskan      917
White                             66237
dtype: int64


While the number of death by date doesn't say much, we can see huge difference in terms of kills between men and women. Man are far more killed than women. 
We can also see that white men are far more killed than any other.

We can't say mush about the huge proportion of white death and we have to dig deeper in the death by race to see if this is proportionaly equivalent for each race or to see if there is some kind of "murder racism"

In [135]:
#read census
census = pd.read_csv("census.csv")
census[:1]

,Id,Year,Id.1,Sex,Id.2,Hispanic Origin,Id.3,Id2,Geography,Total,Race Alone - White,Race Alone - Hispanic,Race Alone - Black or African American,Race Alone - American Indian and Alaska Native,Race Alone - Asian,Race Alone - Native Hawaiian and Other Pacific Islander,Two or More Races
0,cen42010,"April 1, 2010 Census",totsex,Both Sexes,tothisp,Total,0100000US,NaN,United States,308745538,197318956,44618105,40250635,3739506,15159516,674625,6984195


In [136]:
# race killed ratio

census_killed = pd.Series({
    "Asian/Pacific Islander" : census["Race Alone - Asian"] + census["Race Alone - Native Hawaiian and Other Pacific Islander"],
    "Black" : census["Race Alone - Black or African American"], 
    "Native American/Native Alaskan" : census["Race Alone - American Indian and Alaska Native"],
    "Hispanic" : census["Race Alone - Hispanic"],
    "White" : census["Race Alone - White"]
})

(race_group / census_killed) * 100000


race
Asian/Pacific Islander                                  0    8.37431
dtype: float64
Black                             0    57.877348
Name: Race Alone - Black or Afr...
Hispanic                          0    20.220491
Name: Race Alone - Hispanic, dt...
Native American/Native Alaskan    0    24.521956
Name: Race Alone - American Ind...
White                             0    33.568493
Name: Race Alone - White, dtype...
dtype: object

In [137]:
# intent homicide
homicide = gun_death.loc[gun_death["intent"] == "Homicide"]
homicide_by_race = homicide.groupby("race")["race"].count()
(homicide_by_race / census_killed) * 100000

race
Asian/Pacific Islander                                 0    3.530346
dtype: float64
Black                             0    48.471285
Name: Race Alone - Black or Afr...
Hispanic                          0    12.627161
Name: Race Alone - Hispanic, dt...
Native American/Native Alaskan    0    8.717729
Name: Race Alone - American Indi...
White                             0    4.635642
Name: Race Alone - White, dtype:...
dtype: object